In [118]:
from graph_tool.all import *
import os
from datetime import datetime
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as FF

import numpy as np
import pandas as pd

startTime0 = datetime.now()

os.chdir('/home/gu38/Documents/Hadoop FInal Project/')
g = Graph()
g.set_directed(False)
vertex_name = g.new_vertex_property("string")

with open('refined_citations.txt', 'r') as edge_list:
    nodes = {}  # Dictionary of {Node-Name,  Node-Index}
    edges = {}  # Dictionary of {Node-Index, [list of edges from that node]}
    degrees = {} # Dictionary of {Node-Index, Node-Degree}
    locations ={} # Dictionary of {Node-Index, location index in sortedEdges}
    coreLocations = {} # Dictionary of {Node-Degree, Starting index of that degree nodes in sorted {degrees} }
    numDegreeElements = {0:0} #Dictionary of {Node-Degree, Number of nodes with that degree}
    
    i, numedges = 0, 0
    for edge in edge_list:
        vertex_name1, vertex_name2 = edge.strip().split('\t')[0], edge.strip().split('\t')[1]

        if vertex_name1 not in nodes:
            nodes[vertex_name1] = i  # Put the Source vertex of edge in the dictionary
            g.add_vertex()  # Add a new node into the graph with next available index
            vertex_name[g.vertex(i)] = vertex_name1  # Assign the vertex its name
            i = i + 1

        if vertex_name2 not in nodes:
            nodes[vertex_name2] = i
            g.add_vertex()
            vertex_name[g.vertex(i)] = vertex_name2
            i = i + 1

            # The Dictionary is formed. Now let's create the edges:
        if nodes[vertex_name1] in edges:
            edges[nodes[vertex_name1]].append(nodes[vertex_name2])
        else:
            edges[nodes[vertex_name1]] = [nodes[vertex_name2]]
        if nodes[vertex_name2] in edges:
            edges[nodes[vertex_name2]].append(nodes[vertex_name1])
        else:
            edges[nodes[vertex_name2]] = [nodes[vertex_name1]]

        if vertex_name1 != vertex_name2:
            g.add_edge(nodes.get(vertex_name1), nodes.get(vertex_name2))
            numedges += 1

    startTime1 = datetime.now()
    time1 = startTime1 - startTime0
    print("Time till determining unsorted Edges: " + str(time1))

    sortedEdges = sorted(edges.items(), key=lambda item: len(item[1]))

    startTime2 = datetime.now()
    time2 = startTime2 - startTime1
    print("Time elapsed while sorting Edges: " + str(time2))

    maxDegree = len(sortedEdges[len(sortedEdges) - 1][1])
    
    degree, numNode = 0, 0
    
    for vertex in sortedEdges:
        if len(vertex[1]) > degree:
            coreLocations[len(vertex[1])] = len(degrees)
            numDegreeElements[len(vertex[1])] = 1
        else:
            numDegreeElements[len(vertex[1])] += 1
            
        degree = len(vertex[1])
        degrees[vertex[0]] = degree
        locations[vertex[0]] = numNode
        numNode += 1

    print("Final Node count: " + str(numNode))
    print("Final Edge count: " + str(numedges))
    print("Maximum degree of a vertex: " + str(maxDegree))

#-------------------x---------------------Degree Distribution Plot-----------------x-----------------

# degDist = {}
# for item in sortedEdges:
#     deg = len(item[1])
#     if deg not in degDist:
#         degDist[deg] = 1
#     else:
#         degDist[deg] += 1

# # del degDist[1276]
# trace1 = go.Bar(
#     x=list(degDist.keys()), 
#     y=list(degDist.values()), 
#     name='degDist'
# )

# data1 = [trace1]
# layout1 = go.Layout(
#     barmode='group',
#     title = 'Degree Distribution'
# )

# fig1 = go.Figure(data=data1, layout=layout1)
# py.iplot(fig1, filename='degDist')

# graph_draw(g, vertex_font_size=10, vertex_text = vertex_name, output_size=(6000, 6000))

# vprop = kcore_decomposition(g)

#---------------x-------------------Union-Find for Connected Components--------------x------------

startTime2 = datetime.now()

def getRoots(aNeigh):
    def findRoot(aNode, aRoot):
        while aNode != aRoot[aNode][0]:
            aNode = aRoot[aNode][0]
        return (aNode, aRoot[aNode][1])

    myRoot = {}
    for myNode in aNeigh.keys():
        myRoot[myNode] = (myNode, 0)

    for myI in aNeigh:
        for myJ in aNeigh[myI]:
            (myRoot_myI, myDepthMyI) = findRoot(myI, myRoot)
            (myRoot_myJ, myDepthMyJ) = findRoot(myJ, myRoot)
            if myRoot_myI != myRoot_myJ:
                myMin = myRoot_myI
                myMax = myRoot_myJ
                if myDepthMyI > myDepthMyJ:
                    myMin = myRoot_myJ
                    myMax = myRoot_myI
                myRoot[myMax] = (myMax, max(myRoot[myMin][1] + 1, myRoot[myMax][1]))
                myRoot[myMin] = (myRoot[myMax][0], -1)

    myToRet = {}
    for myI in aNeigh:
        if myRoot[myI][0] == myI:
            myToRet[myI] = []
    for myI in aNeigh:
        myToRet[findRoot(myI, myRoot)[0]].append(myI)

    return myToRet


startTime3 = datetime.now()
time3 = startTime3 - startTime2
print("Time elapsed in Union-Find: " + str(time3))

connectedComps = getRoots(edges)
print(len(getRoots(edges)))


#----------------x----------------Core-Decomposition--------------x----------------

startTime4 = datetime.now()
core = {}
maxCore = 0
time5 = datetime.now() - datetime.now()
for vertex in sortedEdges:
    for neighbour in vertex[1]:
        val = degrees[vertex[0]]
        deg = degrees[neighbour]
        
        
        core[vertex[0]] = val
        if val > maxCore:
            maxCore = val
        if degrees[neighbour] > val:

            startTime6 = datetime.now()
            
            indexOld = locations[neighbour]
            indexNew = coreLocations[deg]
            sortedEdges[indexOld], sortedEdges[indexNew] = sortedEdges[indexNew], sortedEdges[indexOld]
            
            locations[neighbour] = indexNew
            locations[sortedEdges[indexOld][0]] = indexOld

            if (deg - 1) not in coreLocations:
                coreLocations[deg - 1] = coreLocations[deg]
                numDegreeElements[deg - 1] = 0

            if numDegreeElements[deg] > 0:
                coreLocations[deg] += 1
                numDegreeElements[deg] -= 1
            else:
                del coreLocations[deg]
#                 print("anybody there")
#                 del numDegreeElements[deg]

            numDegreeElements[deg - 1] += 1

                
            degrees[neighbour] -= 1

            time5 += datetime.now() - startTime6


print("Maximum Core value: " + str(maxCore))
print(time5)

startTime5 = datetime.now()
time4 = startTime5 - startTime4
print("Time elapsed in Core-Decomposition: " + str(time4))

#-------------------x---------------------Core-Decomposition Plot-----------------x-----------------
# conCompDegDist = {}
# for item, conComp in connectedComps.items():
#     deg = len(conComp)
#     if deg not in conCompDegDist:
#         conCompDegDist[deg] = 1
#     else:
#         conCompDegDist[deg] += 1

# del conCompDegDist[242086]

# trace2 = go.Bar(
#     x=list(conCompDegDist.keys()), 
#     y=list(conCompDegDist.values()), 
#     name='conCompDegDist'
# )

# data2 = [trace2]
# layout2 = go.Layout(
#     barmode='group',
#     title = 'Connected Component Degree Distribution'
# )

# fig2 = go.Figure(data=data2, layout=layout2)
# py.iplot(fig2, filename='conCompDegDist')

#--------------------x------------------Peeling (Part #3)-----------------x-------------------

startTime6 = datetime.now()

sortedPeels = sorted(core.items(), key=lambda item: item[1], reverse = True)

curVert = []
edgeLayerValues = {}
peelDegDist = {}

def themPeels(curVert, peelVal):
    startTime = datetime.now()
    peelDegDist[peelVal] = len(curVert)

#     edgeLayerValues[peelVal] = []
#     for vert1 in curVert:
#         for vert2 in sortedEdges1[locations[vert1]][1]:
#             if vert2 in curVert:
#                 if (vert1, vert2) not in edgeLayerValues[peelVal]:
#                     edgeLayerValues[peelVal].append((vert1, vert2))

#                 sortedEdges1[locations[vert1]][1].remove(vert2)
#                 sortedEdges1[locations[vert2]][1].remove(vert1)
    
peelVal = sortedPeels[0][1]
lastElem = sortedPeels[len(sortedPeels) - 1][0]
for peel in sortedPeels:
    if peel[1] == peelVal:
        curVert.append(peel[0])
    else:
        themPeels(curVert, peelVal)
        curVert = [peel[0]]
        peelVal -= 1
    if peel[0] == lastElem:
        themPeels(curVert, peelVal)

startTime7 = datetime.now()
time4 = startTime7 - startTime6
print("Time elapsed in Vertex Peeling: " + str(time4))

#-----------------x-------------- Peeling Done----------------x-----------------


trace3 = go.Bar(
    x=list(peelDegDist.keys()), 
    y=list(peelDegDist.values()), 
    name='peelDegDist'
)

data3 = [trace3]
layout3 = go.Layout(
    barmode='group',
    title = 'Peeling Degree Distribution'
)

fig3 = go.Figure(data=data3, layout=layout3)
py.iplot(fig3, filename='peelDegDist')

#-------------------x---------------------Peeling Plot Done-----------------x-----------------



Time till determining unsorted Edges: 0:15:33.221597
Time elapsed while sorting Edges: 0:00:02.911453
Final Node count: 1559666
Final Edge count: 9805247
Maximum degree of a vertex: 10121
Time elapsed in Union-Find: 0:00:00.000261
25943
Maximum Core value: 244
0:00:24.893909
Time elapsed in Core-Decomposition: 0:00:47.326382
Time elapsed in Vertex Peeling: 4:16:49.135600


In [156]:
#-----------------x--------------No of CC in peel-layers chart----------------x-----------------

sortedConnComps = sorted(connectedComps.items(), key=lambda item: len(item[1]))

pVal = 0            
stackedPeels = {}
for item in sortedConnComps:
    pVal = core[locations[item[0]]]
    
    if pVal not in stackedPeels:
        stackedPeels[pVal] = [[len(item[1]), 1]]
    else: 
        flag = False
        for v in stackedPeels[pVal]:
            if v[0] == len(item[1]):
                v[1] += 1
                flag = True
        if not flag:
            stackedPeels[pVal].append([len(item[1]), 1])
            
del stackedPeels[6][14]  # Removing the odd large value([1495456, 1]). To be displayed separately

trace = [None] * len(stackedPeels)
i = 0
for key, values in stackedPeels.items():
    trace[i] = go.Bar(x=list(v[0] for v in values),y=list(v[1] for v in values),name='Peel size ' + str(key))
    i += 1

trace.append(go.Bar(x=[60],y=[6000],name='1495456 PeelSize for CCsize6'))

data = trace
layout = go.Layout(
    barmode='stack',
    title = 'Connected Componenets in peels'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')

#-----------------x-------------- charting Done----------------x-----------------
